In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from __future__ import annotations

from pydantic import Field

import numpy as np
from qat.model.component import Component, ComponentId, make_refdict
from qat.model.refs import Ref, RefDict, RefList
from qat.model.autopopulate import AutoPopulate

from typing import Optional

In [2]:
class QuantumComponent(Component):
    pass


class PhysicalBaseband(QuantumComponent):
    pass

class PhysicalChannel(QuantumComponent):
    baseband: Ref[PhysicalBaseband]


class PulseChannel(QuantumComponent):
    physical_channel: Ref[PhysicalChannel]
    auxiliary_qubits: RefList[Qubit] = []


class QuantumDevice(QuantumComponent):
    pulse_channels: RefDict[PulseChannel]
    physical_channel: Ref[PhysicalChannel]
    measure_device: Ref[Resonator]


class Resonator(QuantumDevice):
    measure_device: None = None


class Qubit(QuantumDevice):
    measure_device: Ref[Resonator]


In [3]:
class QuantumHardwareModel(AutoPopulate):
    """
    Base class for calibrating our QPU hardware.

    Attributes:
        qubits:
    """

    physical_basebands: dict[ComponentId,PhysicalBaseband] = Field(allow_mutation=False,
                                                        default=dict())

    physical_channels: dict[ComponentId, PhysicalChannel] = Field(allow_mutation=False, 
                                                        default=dict())

    pulse_channels: dict[ComponentId, PulseChannel] = Field(allow_mutation=False, default=dict())
    qubits: dict[ComponentId, Qubit] = Field(allow_mutation=False, default=dict())
    resonators: dict[ComponentId,Resonator] = Field(allow_mutation=False, default=dict())


In [4]:
def make_Hardware(count=10, connections=3, seed=42):
    rng = np.random.default_rng(seed)
    pick = lambda L, size=3: make_refdict(*rng.choice(L, size=size))
    physical_basebands = [PhysicalBaseband() for _ in range(count)]
    physical_channels = [PhysicalChannel(baseband=list(pick(physical_basebands, 1).values())[0]) for _ in range(count)]
    pulse_channels = [PulseChannel(physical_channel=list(pick(physical_channels, 1).values())[0], auxiliary_qubits=[]) for _ in range(count)]
    qubits = [] # finish this
    resonators = [] # finish this

    #for pulse_channel in pulse_channels:
    #    pulse_channel.auxiliary_qubits = list(pick(qubits, 3).values())

    return QuantumHardwareModel(physical_basebands=make_refdict(*physical_basebands),
                                physical_channels=make_refdict(*physical_channels),
                                pulse_channels=make_refdict(*pulse_channels),
                                qubits=make_refdict(*qubits),
                                resonators=make_refdict(*resonators))

In [ ]:
hw1 = make_Hardware()
hw2 = QuantumHardwareModel(**hw1.model_dump())

hw1._deepequal(hw1)